In [43]:
import xlrd
import pandas as pd

In [ ]:
def openBCCfile():
    loc = ('~/projects/bcc_weekly_reports/data/20200120_report.xlsx')

    wb = xlrd.open_workbook(loc)
    sheet = wb.sheet_by_index(2)
    
    print('Rows: {} \nCols: {}'.format(sheet.ncols, sheet.nrows))

In [5]:
loc = ('~/projects/bcc_weekly_reports/data/20200120_report.xlsx')

wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(2)

In [19]:
print('Rows: {} \nCols: {}'.format(sheet.ncols, sheet.nrows))

Rows: 4 
Cols: 24


In [38]:
def importCOAdata(table='totals'):
    '''
    Return a df of one of the 3 tables found in the COA weekly report
    '''
    # Which table to import
    if table == 'totals':
        rows = range(1,3)
    elif table == 'product_category':
        rows = range(4,9)
    elif table == 'fail_category':
        rows = range(9,23)

    data = []
    for row in rows:
        data.append(sheet.row_values(row))

    data_df = pd.DataFrame(data=data[1:], columns=data[0], dtype='int')
    return data_df


In [39]:
importCOAdata(table='totals')

,Certificates of Analysis Received,Tested Batches,,Failed Batches
0,90793,90793,,6124


In [40]:
importCOAdata(table='product_category')

,Tested Batches By Category,Tested Batches,,Failed Batches By Category
0,Flower:,44108,,2401
1,"Inhalable\n(cartridges, waxes, etc.):",31715,,2170
2,"Other\n(edibles, tinctures, topicals, etc.):",14970,,1553
3,Total:,90793,,6124


In [41]:
importCOAdata(table='fail_category')

,*Reasons For Failure,,Failed Batches By Category,
0,Label Claims:,,2379,
1,Pesticides:,,1790,
2,Microbial Impurities:,,929,
3,Residual Solvents:,,361,
4,Homogeneity:,,36,
5,Foreign Material:,,26,
6,Moisture:,,71,
7,Heavy Metals:,,541,
8,Water Activity:,,51,
9,Cannabinoids:,,77,
